# Capstone: Bitcoin: Time Series Forecasting Using Machine Learning

## 1. Data Gathering
---


## Table of Contents
---

1. [Problem Statement](#Problem-Statement)
2. [Executive Summary](#Executive-Summary)
3. [API Scraping](#API-Scraping)
4. [Glassnode API](#Glassnode-API)
5. [Coinmetrics API](#Glassnode-API)

## Problem Statement
   ---

This project examined Bitcoin data in order to create an accurate forecasting model for Bitcoin prices. Both on-chain (Bitcoin protocol data) and off-chain (market data) were used to analyze important factors and their relationship with the asset price. Underlying hypothesis was that fundamental factors of the Bitcoin blockchain may hold predictive power for future prices. The goal of this project can be summarized by the following problem statement:

- Given a  time series of Bitcoin prices and data, can we accurately predict future prices? How accurately?

The data was pulled from two sources: Glassnode and Coinmetrics.

Various models were used for time-series forecasting:

- SARIMA and SARIMAX
- Facebook Prophet
- LSTM RNN

Other models used for regression and feature extration were:

- Linear Regression, Extra Trees and Logistic Regression

Bitcoin -- Important Facts:

- Created in January 3rd, 2009 by Satoshi Nakamoto
    - “Bitcoin: A Peer- to-Peer Electronic Cash System”
- Bitcoin protocol and blockchain:
    - Distributed ledger that maintains the balances of all token trading (data!)
    - Decentralized (no central authority)
    - Transparent
    - Immutable
    - Cheap and Fast
    - Pseudo-anonymous
- Bitcoin token: 
    - Unit of the digital currency that users own and trade. A bitcoin token is held in a bitcoin wallet (wallet address and key)
- Mining and Cryptography:
    - Miners compete to do complex mathematical computations in order to get block reward (bitcoins!) every 10 minutes
    - Block reward is fixed: was 12.5 now is half: 6.25 bitcoins per block
- Bitcoin Halving:
    - Occurs every 210,000 blocks
    - Block reward is halved (July 2016, May 2020, 2024)
    - Hard-coded limit of 21 million bitcoins
- Supply/demand imbalance:
    - Increase in price
- Proven Scarcity: Digital Gold narrative

## Executive Summary
---

To investigate trends and structures in the Bitcoin dataset from the Glassnode and Coinmetrics APIs, we analyzed features that in theory could have impact on prices, investigated trends, created data visualizations to assist in our understanding of the data, then ran time series SARIMA models, Facebook Prophet and deep learning models with LSTM RNNs to inform and communicate our findings.

Data Gathering:

- Created custom functions to pull from Glassnode and Coinmetrics' API
- Ran pull request functions to gather a reasonable amount of data to run insightful models

Exploratory Data Analysis:

- Explored the distribution of key features and their relationship to price
- Analyzed fundamental metrics such as addresses, mining revenue, hashing difficulty and others.
- Used correlation matrices to gain more understanding of potential explanatory and predictive variables

Cleaning and Preprocessing:

- Engineered new columns for title, post and comment lengths
- Properly imputed missing data with correct numerical values
- Merged dataframes into a single dataframe for modeling
- Dropped unecessary columns and otherwise prepared dataset for each respective model

Visualizations:

- Created plots to visualize relationship of potential key variables with price
- Overlayed bitcoin halving dates to infer its impact on price.

Time Series and Deep Learning Models:

- Created univariate and multivariate models for SARIMA(X), Facebook Prophet and LSTM RNNs
- Fine tuned various hyperparameter and network structures for each model
- Used appropriate performance metrics for each model

Conclusions and Further Developments:

- The Facebook Prophet model performed the best amongst the forecasting models
- Bitcoin price is predicted to go up to over $20,000 in the next months
- Would complement the project by further tuning RNN model, use hourly data for bigger dataset and improve feature selection process

## Imports
---

In [285]:
# Standards
import pandas as pd
import numpy as np

# API
import requests
import requests as req
import json
import iso8601

# Automating
import glob
import time
from time import sleep
import datetime as dt
import warnings
import sys

# Supress warnings
import warnings
warnings.filterwarnings('ignore')

import _pickle as pickle

## API Scraping
---

## Glassnode API

### Custom Class to Get Data

Created a custom class that pulls Glassnode data via its API.

In [286]:
# https://github.com/glassnode/glassnode-api-python-client

class GlassnodeClient:

  def __init__(self):
    self._api_key = ''

  @property
  def api_key(self):
    return self._api_key

  def set_api_key(self, value):
    self._api_key = value

  def get(self, url, a='BTC', i='24h', c='native', e=None, s=None, u=None):
    p = dict()
    p['a'] = a
    p['i'] = i
    p['c'] = c

    if s is not None:
      try:
        p['s'] = iso8601.parse_date(s).strftime('%s')
      except ParseError:
        p['s'] = s

    if u is not None:
      try:
        p['u'] = iso8601.parse_date(u).strftime('%s')
      except ParseError:
        p['u'] = s

    p['api_key'] = self.api_key

    r = requests.get(url, params=p)

    try:
       r.raise_for_status()
    except Exception as e:
        print(e)
        print(r.text)

    try:
        df = pd.DataFrame(json.loads(r.text))
        df = df.set_index('t')
        df.index = pd.to_datetime(df.index, unit='s')
        df = df.sort_index()
        col_name = '_'.join(url.split('/')[-2:])
        df.rename(columns = {'v': col_name}, inplace = True)
#         s = df.v
#         s.name = '_'.join(url.split('/')[-2:])
        return df
    except Exception as e:
        print(e)
        
"""
List of possible parameters for API:
a
REQUIRED
string
asset symbol (only: BTC, LTC, BCH)

e
OPTIONAL
string
exchange (see /endpoints)

s
OPTIONAL
integer
since, unix timestamp

u
OPTIONAL
integer
until, unix timestamp

i
OPTIONAL
string
frequency interval, 1h, 10m or 24h (default)

f
OPTIONAL
string
format, csv or json (default)

c
OPTIONAL
string
currency, usd or native (default)
"""

'\nList of possible parameters for API:\na\nREQUIRED\nstring\nasset symbol (only: BTC, LTC, BCH)\n\ne\nOPTIONAL\nstring\nexchange (see /endpoints)\n\ns\nOPTIONAL\ninteger\nsince, unix timestamp\n\nu\nOPTIONAL\ninteger\nuntil, unix timestamp\n\ni\nOPTIONAL\nstring\nfrequency interval, 1h, 10m or 24h (default)\n\nf\nOPTIONAL\nstring\nformat, csv or json (default)\n\nc\nOPTIONAL\nstring\ncurrency, usd or native (default)\n'

In [287]:
gn = GlassnodeClient()

In [288]:
gn.set_api_key('2eea6100-6a2f-44cc-a965-455b897fe07b')

In [289]:
today = dt.datetime.today().strftime("%Y-%m-%d")
today

'2020-11-18'

In [290]:
# ADDRESSES

In [291]:
total_addresses = gn.get('https://api.glassnode.com/v1/metrics/addresses/count',  a='btc', s='2010', u=today)
total_addresses.to_csv('data/total_addresses.csv')
print(total_addresses.shape)
total_addresses.tail(2)

(3974, 1)


,addresses_count
t,
2020-11-16,736497760
2020-11-17,737056640


In [292]:
active_addresses = gn.get('https://api.glassnode.com/v1/metrics/addresses/active_count',  a='btc', s='2010', u=today)
active_addresses.to_csv('data/active_addresses.csv')
print(active_addresses.shape)
active_addresses.head(2)

(3974, 1)


,addresses_active_count
t,
2010-01-01,4
2010-01-02,0


In [293]:
new_addresses = gn.get('https://api.glassnode.com/v1/metrics/addresses/new_non_zero_count',  a='btc', s='2010', u=today)
new_addresses.to_csv('data/new_addresses.csv')
print(new_addresses.shape)
new_addresses.head(2)

(3974, 1)


,addresses_new_non_zero_count
t,
2010-01-01,134
2010-01-02,126


In [294]:
min_10k_addresses = gn.get('https://api.glassnode.com/v1/metrics/addresses/min_10k_count',  a='btc', s='2010', u=today)
min_10k_addresses.to_csv('data/min_10k_addresses.csv')
print(min_10k_addresses.shape)
min_10k_addresses.tail(2)

(3974, 1)


,addresses_min_10k_count
t,
2020-11-16,111
2020-11-17,111


In [295]:
min_1k_addresses = gn.get('https://api.glassnode.com/v1/metrics/addresses/min_1k_count',  a='btc', s='2010', u=today)
min_1k_addresses.to_csv('data/min_1k_addresses.csv')
print(min_1k_addresses.shape)
min_1k_addresses.tail(2)

(3974, 1)


,addresses_min_1k_count
t,
2020-11-16,2237
2020-11-17,2232


In [296]:
non_zero_addresses = gn.get('https://api.glassnode.com/v1/metrics/addresses/non_zero_count',  a='btc', s='2010', u=today)
non_zero_addresses.to_csv('data/non_zero_addresses.csv')
print(non_zero_addresses.shape)
non_zero_addresses.tail(2)

(3974, 1)


,addresses_non_zero_count
t,
2020-11-16,32473955
2020-11-17,32506599


In [297]:
addresses_profit = gn.get('https://api.glassnode.com/v1/metrics/addresses/profit_relative',  a='btc', s='2010', u=today)
addresses_profit.to_csv('data/addresses_profit.csv')
print(addresses_profit.shape)
addresses_profit.tail(2)

(3410, 1)


,addresses_profit_relative
t,
2019-11-16,0.700704
2019-11-17,0.702740


In [298]:
addresses_dist = gn.get('https://api.glassnode.com/v1/metrics/addresses/supply_distribution_relative',  a='btc', s='2010', u=today)
addresses_dist.to_csv('data/addresses_dist.csv')
print(addresses_dist.shape)
addresses_dist.tail(2)

(3608, 1)


,o
t,
2019-11-16,"{'0001_001': 0.001503247800988876, '001_01': 0..."
2019-11-17,"{'0001_001': 0.0015028925893218802, '001_01': ..."


In [299]:
accum_addresses = gn.get('https://api.glassnode.com/v1/metrics/addresses/accumulation_count',  a='btc', s='2010', u=today)
accum_addresses.to_csv('data/accum_addresses.csv')
print(accum_addresses.shape)
accum_addresses.tail(2)

(3608, 1)


,addresses_accumulation_count
t,
2019-11-16,459698
2019-11-17,459975


In [300]:
accum_balance = gn.get('https://api.glassnode.com/v1/metrics/addresses/accumulation_balance',  a='btc', s='2010', u=today)
accum_balance.to_csv('data/accum_balance.csv')
print(accum_balance.shape)
accum_balance.tail(2)

(3608, 1)


,addresses_accumulation_balance
t,
2019-11-16,2.361696e+06
2019-11-17,2.361896e+06


In [301]:
dep_addresses = gn.get('https://api.glassnode.com/v1/metrics/addresses/sending_to_exchanges_count',  a='btc', s='2010', u=today)
dep_addresses.to_csv('data/dep_addresses.csv')
print(dep_addresses.shape)
dep_addresses.tail(2)

(3608, 1)


,addresses_sending_to_exchanges_count
t,
2019-11-16,44963
2019-11-17,44815


In [302]:
with_addresses = gn.get('https://api.glassnode.com/v1/metrics/addresses/receiving_from_exchanges_count',  a='btc', s='2010', u=today)
with_addresses.to_csv('data/with_addresses.csv')
print(with_addresses.shape)
with_addresses.tail(2)

(3608, 1)


,addresses_receiving_from_exchanges_count
t,
2019-11-16,18748
2019-11-17,17005


In [303]:
# TRANSACTIONS

In [304]:
trans_count = gn.get('https://api.glassnode.com/v1/metrics/transactions/count',  a='btc', s='2010', u=today)
trans_count.to_csv('data/trans_count.csv')
print(trans_count.shape)
trans_count.tail(2)

(3974, 1)


,transactions_count
t,
2020-11-16,311317
2020-11-17,347615


In [305]:
trans_rate = gn.get('https://api.glassnode.com/v1/metrics/transactions/rate',  a='btc', s='2010', u=today)
trans_rate.to_csv('data/trans_rate.csv')
print(trans_rate.shape)
trans_rate.tail(2)

(3974, 1)


,transactions_rate
t,
2020-11-16,3.603206
2020-11-17,4.023322


In [306]:
trans_size = gn.get('https://api.glassnode.com/v1/metrics/transactions/size_sum',  a='btc', s='2010', u=today)
trans_size.to_csv('data/trans_size.csv')
print(trans_size.shape)
trans_size.tail(2)

(3974, 1)


,transactions_size_sum
t,
2020-11-16,193367265.0
2020-11-17,201284684.0


In [307]:
transf_adj_vol = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_adjusted_sum',  a='btc', s='2010', u=today)
transf_adj_vol.to_csv('data/transf_adj_vol.csv')
print(transf_adj_vol.shape)
transf_adj_vol.tail(2)

(3974, 1)


,transactions_transfers_volume_adjusted_sum
t,
2020-11-16,500350.562608
2020-11-17,606039.940181


In [308]:
ent_adj_count = gn.get('https://api.glassnode.com/v1/metrics/transactions/entity_adjusted_count',  a='btc', s='2010', u=today)
ent_adj_count.to_csv('data/ent_adj_count.csv')
print(ent_adj_count.shape)
ent_adj_count.tail(2)

(3608, 1)


,transactions_entity_adjusted_count
t,
2019-11-16,211023
2019-11-17,184499


In [309]:
ent_adj_vol = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_entity_adjusted_sum',  a='btc', s='2010', u=today)
ent_adj_vol.to_csv('data/ent_adj_vol.csv')
print(ent_adj_vol.shape)
ent_adj_vol.tail(2)

(3608, 1)


,transactions_transfers_volume_entity_adjusted_sum
t,
2019-11-16,144724.614192
2019-11-17,106854.159103


In [310]:
miner_netflow = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_miners_net',  a='btc', s='2010', u=today)
miner_netflow.to_csv('data/miner_netflow.csv')
print(miner_netflow.shape)
miner_netflow.tail(2)

(3608, 1)


,transactions_transfers_volume_miners_net
t,
2019-11-16,125.416389
2019-11-17,506.360430


In [311]:
exch_inflow_vol = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',  a='btc', s='2010', u=today)
exch_inflow_vol.to_csv('data/exch_inflow_vol.csv')
print(exch_inflow_vol.shape)
exch_inflow_vol.tail(2)

(3381, 1)


,transactions_transfers_volume_to_exchanges_mean
t,
2020-11-16,0.965068
2020-11-17,1.377442


In [312]:
exch_outflow_vol = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_from_exchanges_mean',  a='btc', s='2010', u=today)
exch_outflow_vol.to_csv('data/exch_outflow_vol.csv')
print(exch_outflow_vol.shape)
exch_outflow_vol.tail(2)

(3381, 1)


,transactions_transfers_volume_from_exchanges_mean
t,
2020-11-16,1.423213
2020-11-17,1.266734


In [313]:
# ENTITIES

In [314]:
new_entities = gn.get('https://api.glassnode.com/v1/metrics/entities/new_count',  a='btc', s='2010', u=today)
new_entities.to_csv('data/new_entities.csv')
print(new_entities.shape)
new_entities.tail(2)

(3608, 1)


,entities_new_count
t,
2019-11-16,86250
2019-11-17,74030


In [315]:
entities_growth = gn.get('https://api.glassnode.com/v1/metrics/entities/net_growth_count',  a='btc', s='2010', u=today)
entities_growth.to_csv('data/entities_growth.csv')
print(entities_growth.shape)
entities_growth.tail(2)

(3608, 1)


,entities_net_growth_count
t,
2019-11-16,7620
2019-11-17,3417


In [316]:
whales = gn.get('https://api.glassnode.com/v1/metrics/entities/min_1k_count',  a='btc', s='2010', u=today)
whales.to_csv('data/whales.csv')
print(whales.shape)
whales.tail(2)

(3608, 1)


,entities_min_1k_count
t,
2019-11-16,1733
2019-11-17,1731


In [317]:
active_entities = gn.get('https://api.glassnode.com/v1/metrics/entities/active_count',  a='btc', s='2010', u=today)
active_entities.to_csv('data/active_entities.csv')
print(active_entities.shape)
active_entities.tail(2)

(3608, 1)


,entities_active_count
t,
2019-11-16,202943
2019-11-17,178521


In [318]:
receiving_entities = gn.get('https://api.glassnode.com/v1/metrics/entities/receiving_count',  a='btc', s='2010', u=today)
receiving_entities.to_csv('data/receiving_entities.csv')
print(receiving_entities.shape)
receiving_entities.tail(2)

(3608, 1)


,entities_receiving_count
t,
2019-11-16,156192
2019-11-17,134287


In [319]:
profit_entities = gn.get('https://api.glassnode.com/v1/metrics/entities/profit_relative',  a='btc', s='2010', u=today)
profit_entities.to_csv('data/profit_entities.csv')
print(profit_entities.shape)
profit_entities.tail(2)

(3410, 1)


,entities_profit_relative
t,
2019-11-16,0.736425
2019-11-17,0.737995


In [320]:
entities_dist = gn.get('https://api.glassnode.com/v1/metrics/entities/supply_distribution_relative',  a='btc', s='2010', u=today)
entities_dist.to_csv('data/entities_dist.csv')
print(entities_dist.shape)
entities_dist.tail(2)

(3608, 1)


,o
t,
2019-11-16,"{'0001_001': 0.0011580326648985, '001_01': 0.0..."
2019-11-17,"{'0001_001': 0.0011582004415799, '001_01': 0.0..."


In [321]:
# MARKET

In [322]:
closing_price = gn.get('https://api.glassnode.com/v1/metrics/market/price_usd_close',  a='btc', s='2010', u=today)
closing_price.to_csv('data/closing_price.csv')
print(closing_price.shape)
closing_price.tail(2)

(3777, 1)


,market_price_usd_close
t,
2020-11-16,16709.612284
2020-11-17,17667.381593


In [323]:
ohlc_price = gn.get('https://api.glassnode.com/v1/metrics/market/price_usd_ohlc',  a='btc', s='2010', u=today)
ohlc_price_df = pd.DataFrame()
for i in range(len(ohlc_price['o'])):
    df = pd.DataFrame(ohlc_price.iloc[i]['o'].values()).T
    df.columns = ohlc_price.iloc[i]['o'].keys()
    ohlc_price_df =pd.concat([ohlc_price_df, df])
ohlc_price_df.index = ohlc_price.index
ohlc_price_df.rename(columns={'c':'Close', 'h': 'High', 'l':'Low', 'o':'Open'}, inplace=True)
ohlc_price = ohlc_price_df
print(ohlc_price.shape)
ohlc_price.to_csv('data/ohlc_price.csv')
ohlc_price.tail(2)

(3777, 4)


,Close,High,Low,Open
t,,,,
2020-11-16,16709.612284,16804.409142,15887.108131,15953.998053
2020-11-17,17667.381593,17782.620109,16564.544992,16700.900927


In [324]:
pct_from_ath = gn.get('https://api.glassnode.com/v1/metrics/market/price_drawdown_relative',  a='btc', s='2010', u=today)
pct_from_ath.to_csv('data/pct_from_ath.csv')
print(pct_from_ath.shape)
pct_from_ath.tail(2)

(3777, 1)


,market_price_drawdown_relative
t,
2020-11-16,-0.156588
2020-11-17,-0.108245


In [325]:
realized_price = gn.get('https://api.glassnode.com/v1/metrics/market/price_realized_usd',  a='btc', s='2010', u=today)
realized_price.to_csv('data/realized_price.csv')
print(realized_price.shape)
realized_price.tail(2)

(3974, 1)


,market_price_realized_usd
t,
2020-11-16,6926.408944
2020-11-17,6962.388651


In [326]:
market_cap = gn.get('https://api.glassnode.com/v1/metrics/market/marketcap_usd',  a='btc', s='2010', u=today)
market_cap.to_csv('data/market_cap.csv')
print(market_cap.shape)
market_cap.tail(2)

(3777, 1)


,market_marketcap_usd
t,
2020-11-16,3.031369e+11
2020-11-17,3.162249e+11


In [327]:
realized_mcap = gn.get('https://api.glassnode.com/v1/metrics/market/marketcap_realized_usd',  a='btc', s='2010', u=today)
realized_mcap.to_csv('data/realized_mcap.csv')
print(realized_mcap.shape)
realized_mcap.tail(2)

(3974, 1)


,market_marketcap_realized_usd
t,
2020-11-16,1.284531e+11
2020-11-17,1.291272e+11


In [328]:
mvrv = gn.get('https://api.glassnode.com/v1/metrics/market/mvrv',  a='btc', s='2010', u=today)
mvrv.to_csv('data/mvrv.csv')
print(mvrv.shape)
mvrv.tail(2)

(3777, 1)


,market_mvrv
t,
2020-11-16,2.359903
2020-11-17,2.448941


In [329]:
mvrv_z = gn.get('https://api.glassnode.com/v1/metrics/market/mvrv_z_score',  a='btc', s='2010', u=today)
mvrv_z.to_csv('data/mvrv_z.csv')
print(mvrv_z.shape)
mvrv_z.tail(2)

(3776, 1)


,market_mvrv_z_score
t,
2020-11-16,2.581785
2020-11-17,2.760377


In [330]:
lt_mvrv = gn.get('https://api.glassnode.com/v1/metrics/market/mvrv_more_155',  a='btc', s='2010', u=today)
lt_mvrv.to_csv('data/lt_mvrv.csv')
print(lt_mvrv.shape)
lt_mvrv.tail(2)

(3256, 1)


,market_mvrv_more_155
t,
2019-11-16,1.797264
2019-11-17,1.800986


In [331]:
st_mvrv = gn.get('https://api.glassnode.com/v1/metrics/market/mvrv_less_155',  a='btc', s='2010', u=today)
st_mvrv.to_csv('data/st_mvrv.csv')
print(st_mvrv.shape)
st_mvrv.tail(2)

(3411, 1)


,market_mvrv_less_155
t,
2019-11-16,0.885314
2019-11-17,0.887903


In [332]:
# FUTURES

In [333]:
latest_vol = gn.get('https://api.glassnode.com/v1/metrics/derivatives/futures_volume_daily_latest',  a='btc', s='2010', u=today)
latest_vol.to_csv('data/latest_vol.csv')
print(latest_vol.shape)
latest_vol.tail(2)

(9, 1)


,daily_volume
t,
2020-11-18 21:50:00,35524.378609
2020-11-18 21:50:00,407918.775184


In [334]:
futures_vol = gn.get('https://api.glassnode.com/v1/metrics/derivatives/futures_volume_daily_all_sum',  a='btc', s='2010', u=today)
futures_vol.to_csv('data/futures_vol.csv')
print(futures_vol.shape)
futures_vol.tail(2)

(31, 1)


,derivatives_futures_volume_daily_all_sum
t,
2020-11-16,8.988924e+05
2020-11-17,1.353426e+06


In [335]:
perps_vol = gn.get('https://api.glassnode.com/v1/metrics/derivatives/futures_volume_daily_perpetual_sum',  a='btc', s='2010', u=today)
perps_vol.to_csv('data/perps_vol.csv')
print(perps_vol.shape)
perps_vol.tail(2)

(31, 1)


,derivatives_futures_volume_daily_perpetual_sum
t,
2020-11-16,8.545678e+05
2020-11-17,1.296401e+06


In [336]:
latest_oi = gn.get('https://api.glassnode.com/v1/metrics/derivatives/futures_open_interest_latest',  a='btc', s='2010', u=today)
latest_oi.to_csv('data/latest_oi.csv')
print(latest_oi.shape)
latest_oi.tail(2)

(8, 1)


,open_interest
t,
2020-11-18 21:40:00,3948.299419
2020-11-18 21:40:00,62736.547376


In [337]:
futures_oi = gn.get('https://api.glassnode.com/v1/metrics/derivatives/futures_open_interest_all_sum',  a='btc', s='2010', u=today)
futures_oi.to_csv('data/futures_oi.csv')
print(futures_oi.shape)
futures_oi.tail(2)

(31, 1)


,derivatives_futures_open_interest_all_sum
t,
2020-11-16,209346.065831
2020-11-17,202642.102475


In [338]:
perps_oi = gn.get('https://api.glassnode.com/v1/metrics/derivatives/futures_open_interest_perpetual_sum',  a='btc', s='2010', u=today)
perps_oi.to_csv('data/perps_oi.csv')
print(perps_oi.shape)
perps_oi.tail(2)

(31, 1)


,derivatives_futures_open_interest_perpetual_sum
t,
2020-11-16,165264.190975
2020-11-17,160331.391596


In [339]:
funding_rate = gn.get('https://api.glassnode.com/v1/metrics/derivatives/futures_funding_rate_perpetual',  a='btc', s='2010', u=today)
funding_rate.to_csv('data/funding_rate.csv')
print(funding_rate.shape)
funding_rate.tail(2)

(31, 1)


,derivatives_futures_funding_rate_perpetual
t,
2020-11-16,0.000198
2020-11-17,0.000090


In [340]:
# EXCHANGES

In [341]:
exch_inflow_vol = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_sum',  a='btc', s='2010', u=today)
exch_inflow_vol.to_csv('data/exch_inflow_vol.csv')
print(exch_inflow_vol.shape)
exch_inflow_vol.tail(2)

(3381, 1)


,transactions_transfers_volume_to_exchanges_sum
t,
2020-11-16,43807.318487
2020-11-17,61297.558270


In [342]:
exch_outflow_vol = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_from_exchanges_sum',  a='btc', s='2010', u=today)
exch_outflow_vol.to_csv('data/exch_outflow_vol.csv')
print(exch_outflow_vol.shape)
exch_outflow_vol.tail(2)

(3381, 1)


,transactions_transfers_volume_from_exchanges_sum
t,
2020-11-16,54967.330566
2020-11-17,57884.666159


In [343]:
exch_deposits = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_to_exchanges_count',  a='btc', s='2010', u=today)
exch_deposits.to_csv('data/exch_deposits.csv')
print(exch_deposits.shape)
exch_deposits.tail(2)

(3381, 1)


,transactions_transfers_to_exchanges_count
t,
2020-11-16,45393
2020-11-17,44501


In [344]:
exch_with = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_from_exchanges_count',  a='btc', s='2010', u=today)
exch_with.to_csv('data/exch_with.csv')
print(exch_with.shape)
exch_with.tail(2)

(3381, 1)


,transactions_transfers_from_exchanges_count
t,
2020-11-16,38622
2020-11-17,45696


In [345]:
exch_balances_stacked = gn.get('https://api.glassnode.com/v1/metrics/distribution/balance_exchanges_all',  a='btc', s='2010', u=today)
exch_balances_stacked.to_csv('data/exch_balances_stacked.csv')
print(exch_balances_stacked.shape)
exch_balances_stacked.tail(2)

(3381, 1)


,o
t,
2020-11-16,"{'binance': 284372.01124829, 'bitfinex': 61175..."
2020-11-17,"{'binance': 285621.78752209, 'bitfinex': 62766..."


In [346]:
exch_balance = gn.get('https://api.glassnode.com/v1/metrics/distribution/balance_exchanges',  a='btc', s='2010', u=today)
exch_balance.to_csv('data/exch_balance.csv')
print(exch_balance.shape)
exch_balance.tail(2)

(3381, 1)


,distribution_balance_exchanges
t,
2020-11-16,2.373261e+06
2020-11-17,2.376666e+06


In [347]:
exch_net_flow = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_exchanges_net',  a='btc', s='2010', u=today)
exch_net_flow.to_csv('data/exch_net_flow.csv')
print(exch_net_flow.shape)
exch_net_flow.tail(2)

(3381, 1)


,transactions_transfers_volume_exchanges_net
t,
2020-11-16,-11160.012079
2020-11-17,3412.892111


In [348]:
min_to_exch_stacked = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_miners_to_exchanges_all',  a='btc', s='2010', u=today)
min_to_exch_stacked.to_csv('data/min_to_exch_stacked.csv')
print(min_to_exch_stacked.shape)
min_to_exch_stacked.tail(2)

(2763, 1)


,volumes
t,
2019-11-16,"{'1THash&58COIN': 5.34448167513601, 'BTC.TOP':..."
2019-11-17,"{'1THash&58COIN': 6.07623216973426, 'BTC.TOP':..."


In [349]:
miner_to_exch = gn.get('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_miners_to_exchanges',  a='btc', s='2010', u=today)
miner_to_exch.to_csv('data/miner_to_exch.csv')
print(miner_to_exch.shape)
miner_to_exch.tail(2)

(3014, 1)


,transactions_transfers_volume_miners_to_exchanges
t,
2019-11-16,255.752997
2019-11-17,319.865236


In [350]:
# SUPPLY

In [351]:
circ_supply = gn.get('https://api.glassnode.com/v1/metrics/supply/current',  a='btc', s='2010', u=today)
circ_supply.to_csv('data/circ_supply.csv')
print(circ_supply.shape)
circ_supply.tail(2)

(3974, 1)


,supply_current
t,
2020-11-16,18545412.50
2020-11-17,18546393.75


In [352]:
percent_supply_profit = gn.get('https://api.glassnode.com/v1/metrics/supply/profit_relative',  a='btc', s='2010', u=today)
percent_supply_profit.to_csv('data/percent_supply_profit.csv')
print(percent_supply_profit.shape)
percent_supply_profit.tail(2)

(3776, 1)


,supply_profit_relative
t,
2020-11-16,0.989986
2020-11-17,0.993428


In [353]:
hodl_waves = gn.get('https://api.glassnode.com/v1/metrics/supply/hodl_waves',  a='btc', s='2010', u=today)
hodl_waves.to_csv('data/hodl_waves.csv')
print(hodl_waves.shape)
hodl_waves.tail(2)

(3974, 1)


,o
t,
2020-11-16,"{'1d_1w': 0.0265553078346638, '1m_3m': 0.06735..."
2020-11-17,"{'1d_1w': 0.0265432043491554, '1m_3m': 0.06619..."


In [354]:
rcap_hodl_waves = gn.get('https://api.glassnode.com/v1/metrics/supply/rcap_hodl_waves',  a='btc', s='2010', u=today)
rcap_hodl_waves.to_csv('data/rcap_hodl_waves.csv')
print(rcap_hodl_waves.shape)
rcap_hodl_waves.tail(2)

(3776, 1)


,o
t,
2020-11-16,"{'1d_1w': 0.0605856421889731, '1m_3m': 0.10636..."
2020-11-17,"{'1d_1w': 0.0609485599170486, '1m_3m': 0.10378..."


In [355]:
last_active_more1y = gn.get('https://api.glassnode.com/v1/metrics/supply/active_more_1y_percent',  a='btc', s='2010', u=today)
last_active_more1y.to_csv('data/last_active_more1y.csv')
print(last_active_more1y.shape)
last_active_more1y.tail(2)

(3974, 1)


,supply_active_more_1y_percent
t,
2020-11-16,0.616093
2020-11-17,0.615644


In [356]:
last_active_more3y = gn.get('https://api.glassnode.com/v1/metrics/supply/active_more_3y_percent',  a='btc', s='2010', u=today)
last_active_more3y.to_csv('data/last_active_more3y.csv')
print(last_active_more3y.shape)
last_active_more3y.tail(2)

(3974, 1)


,supply_active_more_3y_percent
t,
2020-11-16,0.323502
2020-11-17,0.323731


In [357]:
adjusted_supply = gn.get('https://api.glassnode.com/v1/metrics/supply/current_adjusted',  a='btc', s='2010', u=today)
adjusted_supply.to_csv('data/adjusted_supply.csv')
print(adjusted_supply.shape)
adjusted_supply.tail(2)

(3974, 1)


,supply_current_adjusted
t,
2020-11-16,1.539046e+07
2020-11-17,1.538678e+07


In [358]:
inflation_rate = gn.get('https://api.glassnode.com/v1/metrics/supply/inflation_rate',  a='btc', s='2010', u=today)
inflation_rate.to_csv('data/inflation_rate.csv')
print(inflation_rate.shape)
inflation_rate.tail(2)

(3974, 1)


,supply_inflation_rate
t,
2020-11-16,0.019876
2020-11-17,0.019499


In [359]:
# UTXO

In [360]:
utxo_created_count = gn.get('https://api.glassnode.com/v1/metrics/blockchain/utxo_created_count',  a='btc', s='2010', u=today)
utxo_created_count.to_csv('data/utxo_created_count.csv')
print(utxo_created_count.shape)
utxo_created_count.tail(2)

(3974, 1)


,blockchain_utxo_created_count
t,
2020-11-16,869288
2020-11-17,974192


In [361]:
utxo_spent_count = gn.get('https://api.glassnode.com/v1/metrics/blockchain/utxo_spent_count',  a='btc', s='2010', u=today)
utxo_spent_count.to_csv('data/utxo_spent_count.csv')
print(utxo_spent_count.shape)
utxo_spent_count.tail(2)

(3974, 1)


,blockchain_utxo_spent_count
t,
2020-11-16,919942
2020-11-17,952316


In [362]:
utxo_created_value_sum = gn.get('https://api.glassnode.com/v1/metrics/blockchain/utxo_created_value_sum',  a='btc', s='2010', u=today)
utxo_created_value_sum.to_csv('data/utxo_created_value_sum.csv')
print(utxo_created_value_sum.shape)
utxo_created_value_sum.tail(2)

(3974, 1)


,blockchain_utxo_created_value_sum
t,
2020-11-16,3.618304e+06
2020-11-17,3.030812e+06


In [363]:
utxo_spent_value_sum = gn.get('https://api.glassnode.com/v1/metrics/blockchain/utxo_spent_value_sum',  a='btc', s='2010', u=today)
utxo_spent_value_sum.to_csv('data/utxo_spent_value_sum.csv')
print(utxo_spent_value_sum.shape)
utxo_spent_value_sum.tail(2)

(3974, 1)


,blockchain_utxo_spent_value_sum
t,
2020-11-16,3.617304e+06
2020-11-17,3.029830e+06


In [364]:
utxo_perc_profit = gn.get('https://api.glassnode.com/v1/metrics/blockchain/utxo_profit_relative',  a='btc', s='2010', u=today)
utxo_perc_profit.to_csv('data/utxo_perc_profit.csv')
print(utxo_perc_profit.shape)
utxo_perc_profit.tail(2)

(3776, 1)


,blockchain_utxo_profit_relative
t,
2020-11-16,0.994615
2020-11-17,0.997520


In [365]:
# FEES

In [366]:
fees_sum = gn.get('https://api.glassnode.com/v1/metrics/fees/volume_sum',  a='btc', s='2010', u=today)
fees_sum.to_csv('data/fees_sum.csv')
print(fees_sum.shape)
fees_sum.tail(2)

(3974, 1)


,fees_volume_sum
t,
2020-11-16,83.457706
2020-11-17,90.541296


In [367]:
fees_mean = gn.get('https://api.glassnode.com/v1/metrics/fees/volume_mean',  a='btc', s='2010', u=today)
fees_mean.to_csv('data/fees_mean.csv')
print(fees_mean.shape)
fees_mean.tail(2)

(3974, 1)


,fees_volume_mean
t,
2020-11-16,0.000268
2020-11-17,0.000260


In [368]:
# MINING

In [369]:
difficulty_latest = gn.get('https://api.glassnode.com/v1/metrics/mining/difficulty_latest',  a='btc', s='2010', u=today)
difficulty_latest.to_csv('data/difficulty_latest.csv')
print(difficulty_latest.shape)
difficulty_latest.tail(2)

(3974, 1)


,mining_difficulty_latest
t,
2020-11-16,75577685065077500000000
2020-11-17,75577685065077500000000


In [370]:
hash_rate_mean = gn.get('https://api.glassnode.com/v1/metrics/mining/hash_rate_mean',  a='btc', s='2010', u=today)
hash_rate_mean.to_csv('data/hash_rate_mean.csv')
print(hash_rate_mean.shape)
hash_rate_mean.tail(2)

(3974, 1)


,mining_hash_rate_mean
t,
2020-11-16,136158304329831000000
2020-11-17,136840305323567000000


In [371]:
miner_revenue = gn.get('https://api.glassnode.com/v1/metrics/mining/revenue_sum',  a='btc', s='2010', u=today)
miner_revenue.to_csv('data/miner_revenue.csv')
print(miner_revenue.shape)
miner_revenue.tail(2)

(3974, 1)


,mining_revenue_sum
t,
2020-11-16,1083.457706
2020-11-17,1071.791296


In [372]:
miner_fee_revenue = gn.get('https://api.glassnode.com/v1/metrics/mining/revenue_from_fees',  a='btc', s='2010', u=today)
miner_fee_revenue.to_csv('data/miner_fee_revenue.csv')
print(miner_fee_revenue.shape)
miner_fee_revenue.tail(2)

(3974, 1)


,mining_revenue_from_fees
t,
2020-11-16,0.077029
2020-11-17,0.084477


In [373]:
miner_block_revenue = gn.get('https://api.glassnode.com/v1/metrics/mining/volume_mined_sum',  a='btc', s='2010', u=today)
miner_block_revenue.to_csv('data/miner_block_revenue.csv')
print(miner_block_revenue.shape)
miner_block_revenue.tail(2)

(3974, 1)


,mining_volume_mined_sum
t,
2020-11-16,1000.00
2020-11-17,981.25


In [374]:
miner_outflow_multiple = gn.get('https://api.glassnode.com/v1/metrics/mining/miners_outflow_multiple',  a='btc', s='2010', u=today)
miner_outflow_multiple.to_csv('data/miner_outflow_multiple.csv')
print(miner_outflow_multiple.shape)
miner_outflow_multiple.tail(2)

(3046, 1)


,mining_miners_outflow_multiple
t,
2019-11-16,1.252816
2019-11-17,0.891787


In [375]:
thermocap = gn.get('https://api.glassnode.com/v1/metrics/mining/thermocap',  a='btc', s='2010', u=today)
thermocap.to_csv('data/thermocap.csv')
print(thermocap.shape)
thermocap.tail(2)

(3776, 1)


,mining_thermocap
t,
2020-11-16,1.893076e+10
2020-11-17,1.894749e+10


In [376]:
marketcap_thermocap_ratio = gn.get('https://api.glassnode.com/v1/metrics/mining/marketcap_thermocap_ratio',  a='btc', s='2010', u=today)
marketcap_thermocap_ratio.to_csv('data/marketcap_thermocap_ratio.csv')
print(marketcap_thermocap_ratio.shape)
marketcap_thermocap_ratio.tail(2)

(3776, 1)


,mining_marketcap_thermocap_ratio
t,
2020-11-16,8.634441e-07
2020-11-17,8.998808e-07


In [377]:
miners_unspent_supply = gn.get('https://api.glassnode.com/v1/metrics/mining/miners_unspent_supply',  a='btc', s='2010', u=today)
miners_unspent_supply.to_csv('data/miners_unspent_supply.csv')
print(miners_unspent_supply.shape)
miners_unspent_supply.tail(2)

(3608, 1)


,mining_miners_unspent_supply
t,
2019-11-16,1.767987e+06
2019-11-17,1.768397e+06


In [378]:
# DISTRIBUTION

In [379]:
balance_1pct_holders = gn.get('https://api.glassnode.com/v1/metrics/distribution/balance_1pct_holders',  a='btc', s='2010', u=today)
balance_1pct_holders.to_csv('data/balance_1pct_holders.csv')
print(balance_1pct_holders.shape)
balance_1pct_holders.tail(2)

(3974, 1)


,distribution_balance_1pct_holders
t,
2020-11-16,0.893518
2020-11-17,0.893712


In [380]:
gini = gn.get('https://api.glassnode.com/v1/metrics/distribution/gini',  a='btc', s='2010', u=today)
gini.to_csv('data/gini.csv')
print(gini.shape)
gini.tail(2)

(3974, 1)


,distribution_gini
t,
2020-11-16,0.987433
2020-11-17,0.987423


In [381]:
herfindahl = gn.get('https://api.glassnode.com/v1/metrics/distribution/herfindahl',  a='btc', s='2010', u=today)
herfindahl.to_csv('data/herfindahl.csv')
print(herfindahl.shape)
herfindahl.tail(2)

(3974, 1)


,distribution_herfindahl
t,
2020-11-16,0.000257
2020-11-17,0.000258


In [382]:
# INDICATORS

In [383]:
sopr = gn.get('https://api.glassnode.com/v1/metrics/indicators/sopr', a = 'btc', s='2010', u=today)
sopr.to_csv('data/sopr.csv')
print(sopr.shape)
sopr.tail(2)

(3777, 1)


,indicators_sopr
t,
2020-11-16,1.006317
2020-11-17,1.015811


In [384]:
adj_sopr = gn.get('https://api.glassnode.com/v1/metrics/indicators/sopr_adjusted', a = 'btc', s='2010', u=today)
adj_sopr.to_csv('data/adj_sopr.csv')
print(adj_sopr.shape)
adj_sopr.tail(2)

(3777, 1)


,indicators_sopr_adjusted
t,
2020-11-16,1.058912
2020-11-17,1.091134


In [385]:
lth_sopr = gn.get('https://api.glassnode.com/v1/metrics/indicators/sopr_more_155', a = 'btc', s='2010', u=today)
lth_sopr.to_csv('data/lth_sopr.csv')
print(lth_sopr.shape)
lth_sopr.tail(2)

(3252, 1)


,indicators_sopr_more_155
t,
2019-11-16,1.265639
2019-11-17,1.219194


In [386]:
sth_sopr = gn.get('https://api.glassnode.com/v1/metrics/indicators/sopr_less_155', a = 'btc', s='2010', u=today)
sth_sopr.to_csv('data/sth_sopr.csv')
print(sth_sopr.shape)
sth_sopr.tail(2)

(3410, 1)


,indicators_sopr_less_155
t,
2019-11-16,0.991642
2019-11-17,0.997702


In [387]:
ntv = gn.get('https://api.glassnode.com/v1/metrics/indicators/nvt', a = 'btc', s='2010', u=today)
ntv.to_csv('data/ntv.csv')
print(ntv.shape)
ntv.tail(2)

(3776, 1)


,indicators_nvt
t,
2020-11-16,37.064838
2020-11-17,30.602593


In [388]:
ntvs = gn.get('https://api.glassnode.com/v1/metrics/indicators/nvts', a = 'btc', s='2010', u=today)
ntvs.to_csv('data/ntvs.csv')
print(ntvs.shape)
ntvs.tail(2)

(3776, 1)


,indicators_nvts
t,
2020-11-16,46.073217
2020-11-17,47.813303


In [389]:
reserve_risk = gn.get('https://api.glassnode.com/v1/metrics/indicators/reserve_risk', a = 'btc', s='2010', u=today)
reserve_risk.to_csv('data/reserve_risk.csv')
print(reserve_risk.shape)
reserve_risk.tail(2)

(3748, 1)


,indicators_reserve_risk
t,
2020-11-16,0.002673
2020-11-17,0.002785


In [390]:
liveliness = gn.get('https://api.glassnode.com/v1/metrics/indicators/liveliness', a = 'btc', s='2010', u=today)
liveliness.to_csv('data/liveliness.csv')
print(liveliness.shape)
liveliness.tail(2)

(3974, 1)


,indicators_liveliness
t,
2020-11-16,0.601580
2020-11-17,0.601637


In [391]:
realized_profit = gn.get('https://api.glassnode.com/v1/metrics/indicators/realized_profit', a = 'btc', s='2010', u=today)
realized_profit.to_csv('data/realized_profit.csv')
print(realized_profit.shape)
realized_profit.tail(2)

(3777, 1)


,indicators_realized_profit
t,
2020-11-16,3.902352e+08
2020-11-17,8.380966e+08


In [392]:
nupl = gn.get('https://api.glassnode.com/v1/metrics/indicators/net_unrealized_profit_loss', a = 'btc', s='2010', u=today)
nupl.to_csv('data/nupl.csv')
print(nupl.shape)
nupl.tail(2)

(3776, 1)


,indicators_net_unrealized_profit_loss
t,
2020-11-16,0.58360
2020-11-17,0.60368


In [393]:
lt_nupl = gn.get('https://api.glassnode.com/v1/metrics/indicators/nupl_more_155', a = 'btc', s='2010', u=today)
lt_nupl.to_csv('data/lt_nupl.csv')
print(lt_nupl.shape)
lt_nupl.tail(2)

(3622, 1)


,indicators_nupl_more_155
t,
2020-11-16,0.664884
2020-11-17,0.682597


In [394]:
puell = gn.get('https://api.glassnode.com/v1/metrics/indicators/puell_multiple', a = 'btc', s='2010', u=today)
puell.to_csv('data/puell.csv')
print(puell.shape)
puell.tail(2)

(3413, 1)


,indicators_puell_multiple
t,
2020-11-16,1.353830
2020-11-17,1.386802


In [395]:
hodled_lost = gn.get('https://api.glassnode.com/v1/metrics/indicators/hodled_lost_coins', a = 'btc', s='2010', u=today)
hodled_lost.to_csv('data/hodled_lost.csv')
print(hodled_lost.shape)
hodled_lost.tail(2)

(3608, 1)


,indicators_hodled_lost_coins
t,
2019-11-16,7.026274e+06
2019-11-17,7.030735e+06


In [396]:
s2f = gn.get('https://api.glassnode.com/v1/metrics/indicators/stock_to_flow_ratio', a = 'btc', s='2010', u=today)
s2f_df = pd.DataFrame()
for i in range(len(s2f['o'])):
    df = pd.DataFrame(s2f.iloc[i]['o'].values()).T
    df.columns = s2f.iloc[i]['o'].keys()
    s2f_df =pd.concat([s2f_df, df])
s2f_df.index = s2f.index
s2f_df.rename(columns={'ratio':'indicators_s2f_ratio'}, inplace=True)
s2f = s2f_df
print(s2f.shape)
s2f.to_csv('data/s2f.csv')
s2f.tail(2)

(3973, 2)


,daysTillHalving,indicators_s2f_ratio
t,,
2020-11-16,1259.0,26083.099434
2020-11-17,1258.0,26225.437940


In [397]:
ent_adj_sopr = gn.get('https://api.glassnode.com/v1/metrics/indicators/sopr_account_based', a = 'btc', s='2010', u=today)
ent_adj_sopr.to_csv('data/ent_adj_sopr.csv')
print(ent_adj_sopr.shape)
ent_adj_sopr.tail(2)

(3410, 1)


,indicators_sopr_account_based
t,
2019-11-16,0.986496
2019-11-17,0.998638


In [398]:
ent_adj_liv = gn.get('https://api.glassnode.com/v1/metrics/indicators/liveliness_account_based', a = 'btc', s='2010', u=today)
ent_adj_liv.to_csv('data/ent_adj_liv.csv')
print(ent_adj_liv.shape)
ent_adj_liv.tail(2)

(3608, 1)


,indicators_liveliness_account_based
t,
2019-11-16,0.610623
2019-11-17,0.610432


In [399]:
ent_adj_mvrv = gn.get('https://api.glassnode.com/v1/metrics/indicators/mvrv_account_based', a = 'btc', s='2010', u=today)
ent_adj_mvrv.to_csv('data/ent_adj_mvrv.csv')
print(ent_adj_mvrv.shape)
ent_adj_mvrv.tail(2)

(3410, 1)


,indicators_mvrv_account_based
t,
2019-11-16,1.450571
2019-11-17,1.461887


In [400]:
ent_adj_nupl = gn.get('https://api.glassnode.com/v1/metrics/indicators/net_unrealized_profit_loss_account_based', a = 'btc', s='2010', u=today)
ent_adj_nupl.to_csv('data/ent_adj_nupl.csv')
print(ent_adj_nupl.shape)
ent_adj_nupl.tail(2)

(3410, 1)


,indicators_net_unrealized_profit_loss_account_based
t,
2019-11-16,0.311655
2019-11-17,0.313732


In [401]:
ent_adj_lth_nupl = gn.get('https://api.glassnode.com/v1/metrics/indicators/nupl_more_155_account_based', a = 'btc', s='2010', u=today)
ent_adj_lth_nupl.to_csv('data/ent_adj_lth_nupl.csv')
print(ent_adj_lth_nupl.shape)
ent_adj_lth_nupl.tail(2)

(3410, 1)


,indicators_nupl_more_155_account_based
t,
2019-11-16,0.484264
2019-11-17,0.485393


In [402]:
diff_ribbon = gn.get('https://api.glassnode.com/v1/metrics/indicators/difficulty_ribbon', a = 'btc', s='2010', u=today)
diff_ribbon_df = pd.DataFrame()
for i in range(len(diff_ribbon['o'])):
    df = pd.DataFrame(diff_ribbon.iloc[i]['o'].values()).T
    df.columns = diff_ribbon.iloc[i]['o'].keys()
    diff_ribbon_df =pd.concat([diff_ribbon_df, df])
diff_ribbon_df.index = diff_ribbon.index
diff_ribbon = diff_ribbon_df
print(diff_ribbon.shape)
diff_ribbon.to_csv('data/diff_ribbon.csv')
diff_ribbon.tail(2)

(3974, 8)


,ma128,ma14,ma200,ma25,ma40,ma60,ma9,ma90
t,,,,,,,,
2020-11-16,77180073821717140000000,72351158730542190000000,73182121023015186000000,78307326694542320000000,80474311801159200000000,81014891970678720000000,72489044471334306000000,78890047893913510000000
2020-11-17,77240894031957930000000,72599353063967990000000,73217299999143180000000,77894917933014630000000,80291637594277270000000,81032845166880780000000,72875124545552210000000,78921019319488000000000


In [403]:
diff_ribbon_comp = gn.get('https://api.glassnode.com/v1/metrics/indicators/difficulty_ribbon_compression', a = 'btc', s='2010', u=today)
diff_ribbon_comp.to_csv('data/diff_ribbon_comp.csv')
print(diff_ribbon_comp.shape)
diff_ribbon_comp.tail(2)

(3974, 1)


,indicators_difficulty_ribbon_compression
t,
2020-11-16,0.043584
2020-11-17,0.041677


In [404]:
hash_ribbon = gn.get('https://api.glassnode.com/v1/metrics/indicators/hash_ribbon', a = 'btc', s='2010', u=today)
hash_ribbon_df = pd.DataFrame()
for i in range(len(hash_ribbon['o'])):
    df = pd.DataFrame(hash_ribbon.iloc[i]['o'].values()).T
    df.columns = hash_ribbon.iloc[i]['o'].keys()
    hash_ribbon_df =pd.concat([hash_ribbon_df, df])
hash_ribbon_df.index = hash_ribbon.index
hash_ribbon = hash_ribbon_df
print(hash_ribbon.shape)
hash_ribbon.to_csv('data/hash_ribbon.csv')
hash_ribbon.tail(2)

(3974, 3)


,crossed,ma30,ma60
t,,,
2020-11-16,0,122524246548091880000,131289406658736650000
2020-11-17,0,123017049700365860000,131182436886566190000


In [405]:
# Blockchain

In [406]:
block_height = gn.get('https://api.glassnode.com/v1/metrics/blockchain/block_height', a = 'btc', s='2010', u=today)
block_height.to_csv('data/block_height.csv')
print(block_height.shape)
block_height.tail(2)

(3974, 1)


,blockchain_block_height
t,
2020-11-16,657265
2020-11-17,657422


In [407]:
block_count = gn.get('https://api.glassnode.com/v1/metrics/blockchain/block_count', a = 'btc', s='2010', u=today)
block_count.to_csv('data/block_count.csv')
print(block_count.shape)
block_count.tail(2)

(3974, 1)


,blockchain_block_count
t,
2020-11-16,160
2020-11-17,157


In [408]:
block_interval_mean = gn.get('https://api.glassnode.com/v1/metrics/blockchain/block_interval_mean', a = 'btc', s='2010', u=today)
block_interval_mean.to_csv('data/block_interval_mean.csv')
print(block_interval_mean.shape)
block_interval_mean.tail(2)

(3974, 1)


,blockchain_block_interval_mean
t,
2020-11-16,536.431250
2020-11-17,552.305732


In [409]:
block_size_mean = gn.get('https://api.glassnode.com/v1/metrics/blockchain/block_size_mean', a = 'btc', s='2010', u=today)
block_size_mean.to_csv('data/block_size_mean.csv')
print(block_size_mean.shape)
block_size_mean.tail(2)

(3974, 1)


,blockchain_block_size_mean
t,
2020-11-16,1.208931e+06
2020-11-17,1.282460e+06


In [410]:
# merging all files into one dataframe
directoryPath = 'data/'
glued_data = pd.DataFrame()
for file_name in glob.glob(directoryPath+'*.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    glued_data = pd.concat([glued_data,x],axis=0)

In [411]:
combined = glued_data.groupby('t').mean()

In [412]:
combined.index = pd.to_datetime(combined.index)
combined = combined.sort_index()

In [413]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94606 entries, 2010-01-01 00:00:00 to 2020-11-18 21:50:00
Columns: 111 entries, market_mvrv_more_155 to indicators_sopr_less_155
dtypes: float64(111)
memory usage: 80.8 MB


In [414]:
((combined.isnull().sum())/len(combined)*100).sort_values(ascending = False)

open_interest                                     99.998943
daily_volume                                      99.998943
derivatives_futures_open_interest_all_sum         99.967233
derivatives_futures_volume_daily_perpetual_sum    99.967233
derivatives_futures_funding_rate_perpetual        99.967233
                                                    ...    
Low                                                1.036932
High                                               1.036932
Close                                              1.036932
market_price_usd_close                             1.036932
Open                                               1.036932
Length: 111, dtype: float64

In [415]:
len(combined.columns)

111

In [416]:
# export for later use
with open("combined_.pickle", 'wb') as fp:
    pickle.dump(combined, fp)

In [417]:
hourly_price = gn.get('https://api.glassnode.com/v1/metrics/market/price_usd_close',  a='btc', s='2010', i='1h' , u=today)
hourly_price.to_csv('data/hourly_price.csv')
print(hourly_price.shape)
hourly_price.tail(2)

(90630, 1)


,market_price_usd_close
t,
2020-11-18 03:00:00,17912.957340
2020-11-18 04:00:00,18393.585252


In [418]:
hourly_ohlc_price = gn.get('https://api.glassnode.com/v1/metrics/market/price_usd_ohlc',  a='btc', s='2010', i='1h', u=today)
ohlc_price_df = pd.DataFrame()
for i in range(len(hourly_ohlc_price['o'])):
    df = pd.DataFrame(hourly_ohlc_price.iloc[i]['o'].values()).T
    df.columns = hourly_ohlc_price.iloc[i]['o'].keys()
    ohlc_price_df =pd.concat([ohlc_price_df, df])
ohlc_price_df.index = hourly_ohlc_price.index
ohlc_price_df.rename(columns={'c':'Close', 'h': 'High', 'l':'Low', 'o':'Open'}, inplace=True)
hourly_ohlc_price = ohlc_price_df
print(hourly_ohlc_price.shape)
hourly_ohlc_price.to_csv('data/hourly_ohlc_price.csv')
hourly_ohlc_price.tail(2)

(90630, 4)


,Close,High,Low,Open
t,,,,
2020-11-18 03:00:00,17912.957340,17912.957340,17638.175145,17674.299609
2020-11-18 04:00:00,18393.585252,18393.585252,17951.328118,17986.879772


In [419]:
with open("hourly_ohlc_price_glassnode.pickle", 'wb') as fp:
    pickle.dump(hourly_ohlc_price, fp)

## Coinmetrics API

Custom class for Coinmetrics API:

In [420]:
# https://github.com/checkmatey/checkonchain/blob/master/general/coinmetrics_api.py
# Import Coinmetrics API
# from checkonchain.general.__init__ import *
import coinmetrics

cm = coinmetrics.Community() # Initialize a reference object, in this case `cm` for the Community API

class Coinmetrics_api:
       
    def __init__(self,asset,begin_timestamp,end_timestamp):
        print('...Fetching Coinmetrics API for '+asset+'...')
        # List all available metrics for BTC.
        self.asset = asset
        self.begin_timestamp=begin_timestamp
        self.end_timestamp=end_timestamp
        self.topcapconst = 35

    def collect_data(self):
        available_data_types = cm.get_available_data_types_for_asset(self.asset)
        metric_list = str((','.join(available_data_types))) #setup complete metric list
        #print(metric_list)
        asset_data = cm.get_asset_data_for_time_range(self.asset, metric_list, self.begin_timestamp, self.end_timestamp)
        return asset_data
    
    def convert_to_pd(self):
        asset_data = Coinmetrics_api.collect_data(self)
        df = coinmetrics.cm_to_pandas(asset_data)
        #Extract Data as column for ease of application
        df.index.name = 'date'
        df.reset_index(inplace=True)
        df['date'] = pd.to_datetime(df['date'])
        #Calc - block height
        df['blk']=df['BlkCnt'].cumsum()
        #Realised Price (Only if present, excludes XMR and ZEC et al.)
        if 'CapRealUSD' in df:
            df['PriceRealUSD'] = df['CapRealUSD']/df['SplyCur']
        #Calc - approx btc block height (Noting BTC blocks were mined from 9/Jan/09)
        df['btc_blk_est'] = (df['date'] - pd.to_datetime(np.datetime64('2009-01-09'),utc=True))
        df['btc_blk_est'] = df['btc_blk_est']/np.timedelta64(1,'D') #convert from timedelta to Days (float)
        df['btc_blk_est'] = df['btc_blk_est']*(24*6) #Note - corrected for neg values in loop below
        #Calc - Daily Issuance
        for i in range(0,len(df.index)):
            #Correct btc_blk_est
            df.loc[i,'btc_blk_est'] = max(0,df.loc[i,'btc_blk_est'])
            if i == 0:
                df.loc[i,'DailyIssuedNtv'] = df.loc[i,'SplyCur']
            else:
                df.loc[i,'DailyIssuedNtv'] = df.loc[i,'SplyCur'] - df.loc[i-1,'SplyCur']
        # Calc - inflation Rate,  S2F, S2F Model, S2F Price
        df['DailyIssuedUSD'] = df['DailyIssuedNtv'] * df['PriceUSD']            
        df['inf_pct_ann'] = df['DailyIssuedNtv']*365/df['SplyCur']
        df['S2F'] = 1/df['inf_pct_ann']     
        df = df.set_index('date')
        df.index = pd.to_datetime(df.index, unit='D')
        df = df.sort_index()
        return df

"""#############################  
    Coinmetrics Community API
BTC = Coinmetrics_api('btc',"2009-01-03","2019-10-07").convert_to_pd()  
################################"""

"""Example Calculations"""
#BTC = Coinmetrics_api('btc',"2009-01-03","2019-10-07").convert_to_pd()
#LTC = Coinmetrics_api('ltc',"2011-10-07",today).convert_to_pd()
#BCH = Coinmetrics_api('bch',"2017-08-01",today).convert_to_pd()
#DASH = Coinmetrics_api('dash',"2014-01-19",today).convert_to_pd()
#DCR = Coinmetrics_api('dcr',"2016-02-08",today).convert_to_pd()
#XMR = Coinmetrics_api('xmr',"2014-04-18",today).convert_to_pd()
#ZEC = Coinmetrics_api('zec',"2016-10-28",today).convert_to_pd()
#ETH = Coinmetrics_api('eth',"2015-07-30",today).convert_to_pd()

"""Time Metrics"""
# date               -datetime64[ns, UTC]
# blk                -ADDED METRIC - block height (Sum of BlkCnt)
# btc_blk_est        -ADDED METRIC - Estimated Bitcoin block-height (Assumes 10min block-times)

"""Blockchain Metrics"""
# AdrActCnt          -Active Addresses
# BlkCnt             -Block Count (Daily)
# BlkSizeByte        -Block Size (Bytes)
# BlkSizeMeanByte    -Block Size Average (Bytes)
# SplyCur            -Current Coin Supply
# inf_pct_ann        -ADDED METRIC - Inflation % annual
# S2F                -ADDED METRIC - Stock-to-Flow Ratio

"""Network Valuation Models"""
# CapMrktCurUSD      - Market Cap
# CapRealUSD         - Realised Cap

"""Network Pricing Models"""
# PriceUSD           - Coin Price USD
# PriceRealised      - Realised Price USD

"""Network Valuation Oscillators"""
# CapMVRVCur         - MVRV Ratio - Market Cap / Realised Cap
# NVTAdj             - NVT Ratio (Adjusted Volume)
# NVTAdj90           - NVT Ratio 90D MA (Adjusted Volume)

"""Transaction Flow Metrics"""
# TxCnt              -Count of Transactions
# TxTfrCnt           -Count of Transaction Transfers (Cleaned Data)
# TxTfrValAdjNtv     -Native Units Transferred (Adjusted Data)
# TxTfrValAdjUSD     -USD Value Transferred (Adjusted Data)
# TxTfrValNtv        -Total Native Units Value Transferred
# TxTfrValUSD        -Total USD Value Transferred
# TxTfrValMeanNtv    -Mean Native Units Transferred
# TxTfrValMeanUSD    -Mean USD Value Transferred
# TxTfrValMedNtv     -Median Native Units Transferred
# TxTfrValMedUSD     -Median USD Value Transferred

"""Miner Metrics"""
# DiffMean          - Average Difficulty
# FeeMeanNtv        - Mean Fee paid in Native Coins (Daily)
# FeeMeanUSD        - Mean Fee paid in USD (Daily)
# FeeMedNtv         - Median Fee paid in Native Coins (Daily)
# FeeMedUSD         - Mean Fee paid in USD (Daily)
# FeeTotNtv         - Total Fees paid in Native Coins (Daily)
# FeeTotUSD         - Total Fees paid in Native Coins (Daily)
# IssContNtv        - Daily Issued Native Coins to Miners/Validators
# IssContPctAnn     - Annualised Inflation Rate
# IssContUSD        - Daily Issued USD to Miners/Validators
# IssTotNtvDaily    - Issued Native Coins to Miners/Validators (Daily)
# IssTotUSD         - Daily Issued USD Value to Miners/Validators (Daily)
# DailyIssuedNtv    - ADDED METRIC - Daily Issued Native Units (Sply_n - Sply_n-1)
# DailyIssuedUSD    - ADDED METRIC - Daily Issued USD Value (Sply_n - Sply_n-1)

"""Market Specific Metrics"""
# ROI1yr
# ROI30d
# VtyDayRet180d
# VtyDayRet30d
# VtyDayRet60d

'Market Specific Metrics'

In [421]:
btc = Coinmetrics_api('btc',"2009-01-03","2020-11-11").convert_to_pd()

...Fetching Coinmetrics API for btc...


In [422]:
btc

,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,...,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d,blk,PriceRealUSD,btc_blk_est,DailyIssuedNtv,DailyIssuedUSD,inf_pct_ann,S2F
date,,,,,,,,,,,,,,,,,,,,,
2009-01-03 00:00:00+00:00,0.0,0.0,0.0,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,0.00,NaN,NaN,NaN
2009-01-04 00:00:00+00:00,0.0,0.0,0.0,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,0.00,NaN,NaN,NaN
2009-01-05 00:00:00+00:00,0.0,0.0,0.0,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,0.00,NaN,NaN,NaN
2009-01-06 00:00:00+00:00,0.0,0.0,0.0,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,0.00,NaN,NaN,NaN
2009-01-07 00:00:00+00:00,0.0,0.0,0.0,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-07 00:00:00+00:00,1055012.0,146.0,198604086.0,1.360302e+06,2.187115,2.755732e+11,1.259985e+11,1.678778e+13,0.000500,7.437259,...,0.026023,0.027211,0.022916,655902.0,6797.245754,622080.0,912.50,1.356555e+07,0.017968,55.655328
2020-11-08 00:00:00+00:00,920529.0,138.0,189145770.0,1.370622e+06,2.269764,2.873323e+11,1.265912e+11,1.678778e+13,0.000384,5.955396,...,0.026128,0.027797,0.023359,656040.0,6828.902721,622224.0,862.50,1.336875e+07,0.016982,58.884464
2020-11-09 00:00:00+00:00,1188873.0,185.0,252934375.0,1.367213e+06,2.237517,2.840099e+11,1.269308e+11,1.678778e+13,0.000440,6.745236,...,0.025851,0.028047,0.023476,656225.0,6846.795593,622368.0,1156.25,1.771354e+07,0.022765,43.927367


In [423]:
with open("btc_coinmetrics.pickle", 'wb') as fp:
    pickle.dump(btc, fp)